# Model Creation from CLI

README: 
Not everything has been tested yet.  
ToDo: Add more explanation

## Installation

#### (General outline) (Don't need to read if on jupyterhub)

1. Install colmap
    - If on jupyterhub: follow steps on this page: https://colmap.github.io/install.html#linux
    - If on windows: download most recent version on this page: https://github.com/colmap/colmap/releases
        - select COLMAP-3.9.1-windows-cuda.zip or COLMAP-3.9.1-windows-no-cuda.zip depending on GPU (nvidea works best with CUDA)

1. (if on windows) Add system variable:
    - https://www.java.com/en/download/help/path.html
    - add directory in colmap that contains the COLMAP.bat file

1. (if on jupyterhub) ??? ToDo


#### Copy paste for jupyterhub (tested and works)

(request multiple CPU's, speeds up compile process immensely)  
Just copy and paste these commands and excecute in the directory where this notebook is located (/project_ghent/something/something/Computervisie-g10/colmap_CLI)

In [ ]:
sudo apt-get update

sudo apt-get install \
    git \
    cmake \
    ninja-build \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libeigen3-dev \
    libflann-dev \
    libfreeimage-dev \
    libmetis-dev \
    libgoogle-glog-dev \
    libgtest-dev \
    libsqlite3-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libceres-dev

In [ ]:
nvidia-smi --query-gpu=compute_cap --format=csv

# use output of this command in when doing cmake
# eg if output is:
#     computecap
#     6.1
# Then insert 61 in the next command
# If other result, change accordingly
# https://github.com/colmap/colmap/issues/2464

In [ ]:
sudo apt-get install gcc-10 g++-10
export CC=/usr/bin/gcc-10
export CXX=/usr/bin/g++-10
export CUDAHOSTCXX=/usr/bin/g++-10

In [ ]:
#Don't execute
"""sudo apt-get install -y \
    nvidia-cuda-toolkit \
    nvidia-cuda-toolkit-gcc"""

In [ ]:
#Don't execute
#sudo cp /opt/conda/lib/libglog.so.2 /usr/lib/x86_64-linux-gnu/libglog.so.2

In [ ]:
git clone https://github.com/colmap/colmap.git
cd colmap
mkdir build
cd build
cmake .. -GNinja -DCMAKE_CUDA_ARCHITECTURES=61
ninja
sudo ninja install

# Takes a long time to run

In [1]:
!colmap --help

# should return a list of available commands if everything installed correctly

/bin/bash: line 1: colmap: command not found


## Creating models

In [ ]:
## Path config here

workspace=../assets/workspace_test
input_imgs=../assets/undistorted_05_short 

### Create sparse model

We are using the automatic reconstructor command, give it an empty workspace to dump the dense model into and give it the directory containing the images to be used for creating the sparse model

In [ ]:
mkdir $workspace

colmap automatic_reconstructor \
    --workspace_path $workspace \
    --image_path $input_imgs \
    --sparse 1 \
    --dense 0 \
    --single_camera 1

# dense -> 0 to avoid double work, we want more control over the process of turning sparse into a dense model
# single_camera -> only one camera is used

In [ ]:
## Convert results to txt files (optional)

mkdir $workspace/out_dense

colmap model_converter --input_path $workspace/sparse/0 \
    --output_path $workspace/sparse_txt/0 \
    --output_type TXT

### Align model orientation -> axis is more logical

##### Sparse model

In [ ]:
!mkdir ../assets/workspace/sparse_aligned

!colmap model_orientation_aligner \
        --input_path ../assets/workspace/sparse/0 \
        --output_path ../assets/workspace/sparse_aligned \
        --image_path ../assets/undistorted_05_short/
        --method MANHATTAN-WORLD

# I tried the IMAGE-ORIENTATION method aswel but it doesn't work very well
# Colmap has a weird quirk where the Y-axis points downwards. This is by design appearantly.
# https://medium.com/red-buffer/mastering-3d-spaces-a-comprehensive-guide-to-coordinate-system-conversions-in-opencv-colmap-ef7a1b32f2df

In [4]:
# Convert results to txt files (optional)
!mkdir ../assets/workspace/sparse_aligned_man_txt
!colmap model_converter --input_path $workspace/sparse_aligned_man --output_path $workspace/sparse_aligned_man_txt --output_type TXT

##### Dense model

In [ ]:
"""!mkdir ../assets/workspace/out_dense

!colmap model_orientation_aligner --input_path ../assets/workspace/dense/sparse --output_path ../assets/workspace/out_dense --image_path ../assets/workspace/dense/images
"""

ToDo: add explenation here of how to open rotated pointclouds in colmap

### Create dense model

In [ ]:
# todo: Add CLI-commands here

!colmap image_undistorter --help
!colmap patch_match_stereo --help
!colmap stereo_fusion --help

In [ ]:
colmap image_undistorter \
    --image_path ../assets/undistorted_05_short/ \
    --input_path ../assets/workspace/sparse_aligned \
    --output_path ../assets/workspace/dense \
    --output_type COLMAP \
    --max_image_size 2000

In [ ]:
colmap patch_match_stereo \
    --workspace_path ../assets/workspace/dense \
    --workspace_format COLMAP \
    --PatchMatchStereo.geom_consistency true

In [2]:
colmap stereo_fusion \
    --workspace_path ../assets/workspace/dense \
    --workspace_format COLMAP \
    --input_type geometric \
    --output_path ../assets/workspace/dense/fused.ply

In [5]:
mkdir ../assets/workspace/dense/sparse_txt

!colmap model_converter --input_path ../assets/workspace/dense/sparse --output_path ../assets/workspace/dense_txt --output_type TXT

In [7]:
!colmap model_converter --input_path ../assets/workspace/dense/0/sparse --output_path ../assets/workspace/dense/0/sparse_txt --output_type TXT

In [ ]:

DATASET_PATH=../assets/workspace_more_points

IMG_PATH=../assets/undistorted_0_short

colmap feature_extractor \
   --database_path $DATASET_PATH/database.db \
   --image_path $IMG_PATH \
    --SiftExtraction.domain_size_pooling true \
    --SiftExtraction.estimate_affine_shape true 

colmap exhaustive_matcher \
   --database_path $DATASET_PATH/database.db \
    --SiftMatching.guided_matching true

mkdir $DATASET_PATH/sparse

colmap mapper \
    --database_path $DATASET_PATH/database.db \
    --image_path $IMG_PATH \
    --output_path $DATASET_PATH/sparse